# Projeto 2-2: Contagem de Pessoas

### Guilherme Zaborowsky Graicer

**Motivação**: http://www.cvg.reading.ac.uk/PETS2009/a.html#s1

Objetivo: Vocês devem tentar realizar a contagem de pessoas no dataset S1.L1 utilizando o modelo de codebook. As imagens foram distribuídas via pendrive.

Requisitos:
  * Tem que usar o modelo de codebook do artigo.
  * A partir da separação do foreground, vocês podem utilizar qualquer método para realizar a contagem, incluindo Clustering ou Machine Learning.
  * O Repositório no Git deve conter além do código fonte, os dados do codebook treinado salvo com Pickle e um relatório do projeto.
  * O relatório deve conter a metodologia utilizada (incluindo treinamento) e comentário sobre os resultados obtidos.

### Entrega: 03/Out 23:59 via GitHub.

### Rubrica do Projeto:

    I. Não entregou ou entregou apenas um rascunho.
    D. Implementou parcialmente o artigo ou não realizou a contagem.
    C. Implementou a sessão 2 do artigo e realizou uma contagem de forma simples.
    B. Cumpriu a rubrica C com a implementação completa do codebook.
    A. Além de cumprir a rubrica B, realiza a contagem com uma técnica elaborada.
    
    -1 Conceito se o relatório estiver ausente ou insatisfatório.

In [1]:
%reset -f
%matplotlib inline

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import os

In [2]:
class codeword:
    def __init__(self, r, g, b, i_min, i_max, f, lambida, p, q):
        self.rgb = [r,g,b]
        self.aux = [i_min, i_max, f, lambida, p, q]

e1 = 12
e2 = 8
beta = 1.3 #typically entre 1.1 and 1.5
alpha = 0.5 # typically entre 0.4 and 0.7

#Calcula a distancia de cor entre um pixel e um codeword
def color_dist(x, v):
    x2 = x[0]**2 + x[1]**2 + x[2]**2
    v2 = v[0]**2 + v[1]**2 + v[2]**2
    xv2 = (v[0]*x[0] + v[1]*x[1] + v[2]*v[2])**2
    if v2 ==0:
        p2 = 0
    else:
        p2 = xv2 / v2
    
    return np.sqrt(abs(x2 - p2))

# Retorna True se a intensidade do pixel (I) está aceitavel para um codeword
# com certa intensidade minima e maxima (Imin e Imax). Retorna False caso contrário.
def bright(I, Imin, Imax):
    Ilow = Imax*alpha
    Ihi = min((beta*Imax),(Imin/alpha))
    if I <= Ihi and I >= Ilow:
        return True
    return False
    

def codebook_update(x, codebook, t):
    achou = False
    I = np.sqrt(x[0]**2+x[1]**2+x[2]**2)
    for c in codebook:
        cd = color_dist(x, c.rgb)
        if cd <= e1 and bright(I, c.aux[0], c.aux[1]):
            achou = True
            # ATUALIZA A CODEWORD
            r_new = (c.aux[2]*c.rgb[0]+x[0])/(c.aux[2]+1)
            g_new = (c.aux[2]*c.rgb[1]+x[1])/(c.aux[2]+1)
            b_new = (c.aux[2]*c.rgb[2]+x[2])/(c.aux[2]+1)
            c.rgb = [r_new, g_new, b_new]
            c.aux = [min(I, c.aux[0]), max(I, c.aux[1]), (c.aux[2]+1), max(c.aux[3], (t-c.aux[5])), c.aux[4], t]
            break
    if achou == False:
        #CRIA NOVO CODEWORD
        c_new = codeword(x[0], x[1], x[2], I, I, 1, t-1, t, t)
        codebook.append(c_new)
        
    achou = False

    
def create_matrix(rows, cols):
    m = []
    for i in range(rows):
        m.append([])
        for j in range(cols):
            m[i].append([])
    return m
        

In [3]:


codebook_matrix = None

T = 10

directory = "img/BG/Time_13-19/"
images = os.listdir(directory)

for t in tqdm(range(1,(T+1)), unit="Image"):
#     img = cv.imread(("img/BG/Time_13-19/0000000" + str(t) + ".jpg"))
    img = cv.imread(directory + images[t], 1)
    img = img.astype("float")
    rows, cols, cor = img.shape
    
    if t == 1:
        codebook_matrix = create_matrix(rows, cols)
    
    for j in range(cols):
        for i in range(rows):
            codebook_update(img[i][j], codebook_matrix[i][j], t)
            
    for j in range(cols):
        for i in range(rows):
            cb = codebook_matrix[i][j]
            for c in cb:
                c.aux[3] = max(c.aux[3], (T-c.aux[5]+c.aux[4]-1))
            

            
    

100%|██████████| 10/10 [02:11<00:00, 16.67s/Image]


In [4]:
pickle.dump(codebook_matrix, open("codebook_salvo", "wb"))

MemoryError: 

In [5]:
qtos = []
for j in range(cols):
    for i in range(rows):
        cb = codebook_matrix[i][j]
        qtos.append(len(cb))
        
for i in range(1,(T+1)):      
    print("Codebooks com " + str(i) + " words: " + str(qtos.count(i)))

Codebooks com 1 words: 3565
Codebooks com 2 words: 13808
Codebooks com 3 words: 28509
Codebooks com 4 words: 38539
Codebooks com 5 words: 38580
Codebooks com 6 words: 45110
Codebooks com 7 words: 70877
Codebooks com 8 words: 91952
Codebooks com 9 words: 78028
Codebooks com 10 words: 33400


In [ ]:
# Constroi o codebook para 1 pixel em varias imagens
# def codebook_construction(X)
#     codebook = []
#     for i in range(len(X)):
#         t = i+1
#         achou = False
#         I = np.sqrt(X[i][0]**2+X[i][1]**2+X[i][2]**2)
#         for c in codebook:
#             cd = color_dist(X[i], c.rgb)
#             if cd <= e1 and bright(I, c.aux["i_min"], c.aux["i_max"]):
#                 achou = True
#                 # ATUALIZA A CODEWORD
#                 r_new = (c.aux["f"]*c.rgb[0]+X[i][0])/(c.aux["f"]+1)
#                 g_new = (c.aux["f"]*c.rgb[1]+X[i][1])/(c.aux["f"]+1)
#                 b_new = (c.aux["f"]*c.rgb[2]+X[i][2])/(c.aux["f"]+1)
#                 c.rgb = [r_new, g_new, b_new]
#                 c.aux = [min(I, c.aux["i_min"]), max(I, c.aux["i_max"]), (c.aux["f"]+1), max(c.aux["lambida"], (t-c.aux["q"])), c.aux["p"], t]
#                 break
#         if !achou:
#             #CRIA NOVO CODEWORD
#             c_new = codeword(X[i][0], X[i][1], X[i][2], I, I, 1, t-1, t, t)
#             codebook.append(c_new)
            
#         for c in codebook:
#             c.aux["lambida"] = max(c.aux["lambida"], (len(X)-c.aux["q"]+c.aux["p"]-1))
#         achou = False

#     return codebook
